# 📘 Visualizing Linear Regression with Gradient Descent (Real-Scale)

This notebook demonstrates how **linear regression** is applied to actual insurance data using **gradient descent**, with **no normalization** applied to the features.

---

## 🔍 What You'll Learn

- 📈 How a regression line is fit to real-world data (`age` vs `charges`)
- ⚙️ How the parameters `w` (slope) and `b` (intercept) are updated over time using **gradient descent**
- 🧮 How the model minimizes **Mean Squared Error (MSE)** across training epochs
- 📊 Dual visualizations per epoch:
  - **Left:** Data points and current regression line
  - **Right:** MSE vs `w` curve, including:
    - 🔴 Current `(w, MSE)` point
    - 🟢 Tangent line showing the slope at that point
    - 🔻 Gradient descent path (history of weights)

---

## 📐 Why Use Real (Non-Normalized) Scale?

- Helps you understand real regression values like:  
  `charges = 2600 × age + 2300`
- Makes it easier to communicate results to non-technical stakeholders
- Forces more realistic handling of learning rate and convergence speed

---

## 🧪 Tips

- Try adjusting the `learning rate (lr)` and `epochs` to see how fast/slow it converges
- View the tangent line to understand the slope of the loss surface
- Modify the plotting range or animate it into a `.gif` for presentations!

---

✅ Built for learning, testing, and sharing real-world regression intuition.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Load data
df = pd.read_csv("data/insurance.csv")
X = df["age"].values
y = df["charges"].values

# Parameter awal
w, b = 0.0, 0.0
lr = 1e-4  # learning rate lebih kecil karena charges besar
epochs = 300
history = []
m = len(X)

# Training manual
for epoch in range(epochs):
    y_pred = w * X + b
    error = y_pred - y
    mse = np.mean(error**2)

    history.append({
        "epoch": epoch,
        "mse": mse,
        "w": w,
        "b": b,
        "y_pred": y_pred.copy()
    })

    dw = (2 / m) * np.sum(error * X)
    db = (2 / m) * np.sum(error)
    w -= lr * dw
    b -= lr * db

# Static range for w (skala data asli)
w_range = np.linspace(-500, 500, 200)

# Derivatif numerik
def numerical_derivative(w_vals, mse_vals, w_point):
    idx = np.searchsorted(w_vals, w_point)
    if idx <= 0: idx = 1
    if idx >= len(w_vals) - 1: idx = len(w_vals) - 2
    x0, x1 = w_vals[idx - 1], w_vals[idx + 1]
    y0, y1 = mse_vals[idx - 1], mse_vals[idx + 1]
    return (y1 - y0) / (x1 - x0)

# Visualisasi
def plot_true_scale(epoch):
    record = history[epoch]
    w_curr = record["w"]
    b_curr = record["b"]
    y_pred = record["y_pred"]

    # MSE curve
    mse_curve = []
    for w_val in w_range:
        y_hat = w_val * X + b_curr
        mse = np.mean((y_hat - y)**2)
        mse_curve.append(mse)

    mse_curr = np.mean((w_curr * X + b_curr - y)**2)
    slope = numerical_derivative(w_range, mse_curve, w_curr)

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Left: Fit line
    axes[0].scatter(X, y, label="Actual", alpha=0.6)
    axes[0].plot(X, y_pred, color="red", label="Prediction")
    axes[0].set_title(f"Fit Line - Epoch {epoch}")
    axes[0].set_xlabel("Age")
    axes[0].set_ylabel("Charges")
    axes[0].legend()
    axes[0].grid(True)

    # Tampilkan persamaan regresi
    x_mid = 40
    y_mid = w_curr * x_mid + b_curr
    axes[0].annotate(f"y = {b_curr:.2f} + {w_curr:.2f}×x",
                     xy=(x_mid, y_mid),
                     xytext=(x_mid + 5, y_mid + 5000),
                     arrowprops=dict(arrowstyle="->", color='red'),
                     fontsize=10, color='red',
                     bbox=dict(boxstyle="round", facecolor="white", alpha=0.7))

    # Right: MSE vs w
    axes[1].plot(w_range, mse_curve, label="MSE Curve")
    axes[1].plot([w_curr], [mse_curr], 'ro', label="Current (w, MSE)")

    # Tangent line
    w_tangent = np.linspace(w_curr - 100, w_curr + 100, 100)
    mse_tangent = slope * (w_tangent - w_curr) + mse_curr
    axes[1].plot(w_tangent, mse_tangent, 'g--', label="Tangent Line")

    # GD path
    past_ws = [h["w"] for h in history[:epoch+1]]
    past_mses = [np.mean((h["w"] * X + h["b"] - y)**2) for h in history[:epoch+1]]
    axes[1].plot(past_ws, past_mses, 'ro-', alpha=0.5)

    axes[1].set_title("Gradient Descent Path on MSE vs w")
    axes[1].set_xlabel("w")
    axes[1].set_ylabel("MSE")
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

# Interaktif slider
interact(plot_true_scale, epoch=IntSlider(min=0, max=epochs-1, step=1, value=0));


interactive(children=(IntSlider(value=0, description='epoch', max=299), Output()), _dom_classes=('widget-inter…